In [ ]:
import os
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

# Deep Learning - TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, 
                                     Dropout, BatchNormalization, 
                                     GlobalAveragePooling2D, Input)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, InceptionV3

# Metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# %%
# Set random seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

In [ ]:
BASE_PATH = '/kaggle/input/object-classification'
TRAIN_DIR = os.path.join(BASE_PATH, 'train')  # Changed to lowercase
VALID_DIR = os.path.join(BASE_PATH, 'valid')  # Changed to lowercase
TEST_DIR = os.path.join(BASE_PATH, 'test')    # Changed to lowercase

# Class names
CLASS_NAMES = ['bird', 'drone']

print(f"Base Path: {BASE_PATH}")
print(f"Train Directory: {TRAIN_DIR}")
print(f"Valid Directory: {VALID_DIR}")
print(f"Test Directory: {TEST_DIR}")


In [ ]:
def count_images(base_path):
    """Count images in each class for all splits"""
    data = []
    for split in ['train', 'valid', 'test']:   # <-- FIXED lowercase
        split_path = os.path.join(base_path, split)
        if os.path.exists(split_path):
            for cls in os.listdir(split_path):
                cls_path = os.path.join(split_path, cls)
                if os.path.isdir(cls_path):
                    count = len([
                        f for f in os.listdir(cls_path)
                        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
                    ])
                    data.append({
                        'Split': split.upper(),  # Better for display
                        'Class': cls,
                        'Count': count
                    })
    return pd.DataFrame(data)


BASE_PATH = "/kaggle/input/object-classification"
df_counts = count_images(BASE_PATH)

print("Dataset Distribution:")
print(df_counts.pivot(index='Class', columns='Split', values='Count'))


In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, split in enumerate(['TRAIN', 'VALID', 'TEST']):
    split_data = df_counts[df_counts['Split'] == split]
    colors = ['#3498db', '#e74c3c']
    
    axes[idx].bar(split_data['Class'], split_data['Count'], color=colors)
    axes[idx].set_title(f'{split} Set Distribution', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Class')
    axes[idx].set_ylabel('Number of Images')

    # ---- LABELS INSIDE BARS ----
    for i, (cls, count) in enumerate(zip(split_data['Class'], split_data['Count'])):
        axes[idx].text(
            i, 
            count * 0.5,          # middle height
            str(count),
            ha='center',
            va='center',
            fontweight='bold',
            color='white' if count > 20 else 'black'   # white text if bar is tall
        )

plt.tight_layout()
plt.savefig('dataset_distribution.png', dpi=160, bbox_inches='tight')
plt.show()


In [ ]:
train_data = df_counts[df_counts['Split'] == 'TRAIN']
total_train = train_data['Count'].sum()
print("\nClass Imbalance Analysis (Training Set):")
for _, row in train_data.iterrows():
    percentage = (row['Count'] / total_train) * 100
    print(f"  {row['Class']}: {row['Count']} images ({percentage:.1f}%)")


In [ ]:
# Visualize sample images from each class
def display_sample_images(data_dir, class_names, samples_per_class=5):
    """Display sample images from each class"""
    fig, axes = plt.subplots(len(class_names), samples_per_class, 
                             figsize=(3*samples_per_class, 3*len(class_names)))
    
    for i, cls in enumerate(class_names):
        cls_path = os.path.join(data_dir, cls)
        images = os.listdir(cls_path)
        sample_images = random.sample(images, min(samples_per_class, len(images)))
        
        for j, img_name in enumerate(sample_images):
            img_path = os.path.join(cls_path, img_name)
            img = Image.open(img_path)
            axes[i, j].imshow(img)
            axes[i, j].axis('off')
            if j == 0:
                axes[i, j].set_title(f'{cls.upper()}', fontsize=12, fontweight='bold')
    
    plt.suptitle('Sample Images from Training Set', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('sample_images.png', dpi=150, bbox_inches='tight')
    plt.show()

display_sample_images(TRAIN_DIR, CLASS_NAMES)

In [ ]:
# Check image properties (size, channels)
def analyze_image_properties(data_dir, num_samples=50):
    """Analyze image dimensions and properties"""
    widths, heights = [], []
    
    for cls in os.listdir(data_dir):
        cls_path = os.path.join(data_dir, cls)
        if os.path.isdir(cls_path):
            images = os.listdir(cls_path)[:num_samples]
            for img_name in images:
                img_path = os.path.join(cls_path, img_name)
                try:
                    img = Image.open(img_path)
                    widths.append(img.size[0])
                    heights.append(img.size[1])
                except:
                    pass
    
    print("Image Dimension Statistics:")
    print(f"  Width  - Min: {min(widths)}, Max: {max(widths)}, Mean: {np.mean(widths):.0f}")
    print(f"  Height - Min: {min(heights)}, Max: {max(heights)}, Mean: {np.mean(heights):.0f}")
    
    return widths, heights

widths, heights = analyze_image_properties(TRAIN_DIR)


In [ ]:
# Configuration
IMG_SIZE = (224, 224)  # Standard size for most pretrained models
BATCH_SIZE = 32
NUM_CLASSES = 2

print(f"Image Size: {IMG_SIZE}")
print(f"Batch Size: {BATCH_SIZE}")
print(f"Number of Classes: {NUM_CLASSES}")


In [ ]:
# Create ImageDataGenerators

# Training data generator WITH augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,                    # Normalize pixel values to [0, 1]
    rotation_range=30,                 # Random rotation
    width_shift_range=0.2,             # Horizontal shift
    height_shift_range=0.2,            # Vertical shift
    shear_range=0.2,                   # Shear transformation
    zoom_range=0.2,                    # Random zoom
    horizontal_flip=True,              # Random horizontal flip
    vertical_flip=True,                # Random vertical flip
    brightness_range=[0.8, 1.2],       # Brightness adjustment
    fill_mode='nearest'                # Fill mode for new pixels
)

# Validation & Test data generator - ONLY rescaling (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# %%
# Create data generators
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=True,
    seed=SEED
)

valid_generator = val_test_datagen.flow_from_directory(
    VALID_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

# %%
# Print class indices
print("Class Indices:", train_generator.class_indices)
print(f"\nTraining samples: {train_generator.samples}")
print(f"Validation samples: {valid_generator.samples}")
print(f"Test samples: {test_generator.samples}")


In [ ]:
# Visualize augmented images
def visualize_augmentation(data_dir, class_name, num_augmented=6):
    """Visualize original vs augmented images"""
    cls_path = os.path.join(data_dir, class_name)
    img_name = random.choice(os.listdir(cls_path))
    img_path = os.path.join(cls_path, img_name)
    
    # Load original image
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)
    
    # Generate augmented images
    fig, axes = plt.subplots(2, 4, figsize=(16, 8))
    axes = axes.flatten()
    
    # Original image
    axes[0].imshow(img)
    axes[0].set_title('ORIGINAL', fontsize=10, fontweight='bold')
    axes[0].axis('off')
    
    # Augmented images
    aug_iter = train_datagen.flow(img_array, batch_size=1)
    for i in range(1, 8):
        aug_img = next(aug_iter)[0]
        axes[i].imshow(aug_img)
        axes[i].set_title(f'Augmented {i}', fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle(f'Data Augmentation Examples - {class_name.upper()}', 
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('augmentation_examples.png', dpi=150, bbox_inches='tight')
    plt.show()

visualize_augmentation(TRAIN_DIR, 'bird')

In [ ]:
def build_custom_cnn():
    """Build a custom CNN architecture from scratch"""
    model = Sequential([
        # Input Layer
        Input(shape=(224, 224, 3)),
        
        # Convolutional Block 1
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Convolutional Block 2
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Convolutional Block 3
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Convolutional Block 4
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        
        # Fully Connected Layers
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        
        # Output Layer
        Dense(1, activation='sigmoid')
    ])
    
    return model

# %%
# Build and compile Custom CNN
custom_cnn = build_custom_cnn()

custom_cnn.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Display model summary
print("=" * 60)
print("CUSTOM CNN ARCHITECTURE")
print("=" * 60)
custom_cnn.summary()


In [ ]:
def build_transfer_model(base_model_name='resnet50', trainable=False):
    
    # Select base model
    if base_model_name == 'resnet50':
        base_model = ResNet50(
            weights='imagenet', 
            include_top=False, 
            input_shape=(224, 224, 3)
        )
    elif base_model_name == 'inceptionv3':
        base_model = InceptionV3(
            weights='imagenet', 
            include_top=False, 
            input_shape=(224, 224, 3)
        )
    else:
        raise ValueError(f"Unknown model: {base_model_name}")
    
    # Freeze/Unfreeze base model
    base_model.trainable = trainable
    
    # Build the model
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs, outputs)
    
    return model, base_model

# %%
# Build Transfer Learning Models

# ResNet50
print("Building ResNet50...")
resnet_model, resnet_base = build_transfer_model('resnet50')
resnet_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# InceptionV3
print("Building InceptionV3...")
inceptionv3_model, inceptionv3_base = build_transfer_model('inceptionv3')
inceptionv3_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n✅ All transfer learning models built successfully!")

# %%
# Display ResNet50 model summary
print("=" * 60)
print("RESNET50 TRANSFER LEARNING ARCHITECTURE")
print("=" * 60)
resnet_model.summary()

In [ ]:
# Define callbacks
def get_callbacks(model_name):
    """Create callbacks for training"""
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=10,
            restore_best_weights=True,
            verbose=1
        ),
        ModelCheckpoint(
            f'models/best_{model_name}.keras',
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        ),
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.2,
            patience=5,
            min_lr=1e-7,
            verbose=1
        )
    ]
    return callbacks

# Create models directory
os.makedirs('models', exist_ok=True)

EPOCHS = 30


In [ ]:
# Train Custom CNN
print("\n" + "=" * 60)
print("TRAINING CUSTOM CNN")
print("=" * 60)

history_cnn = custom_cnn.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS,
    callbacks=get_callbacks('custom_cnn'),
    verbose=1
)


In [ ]:
# %%
# Train ResNet50
print("\n" + "=" * 60)
print("TRAINING RESNET50 (Feature Extraction)")
print("=" * 60)

history_resnet = resnet_model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    callbacks=get_callbacks('resnet50'),
    verbose=1
)


In [ ]:
# Train InceptionV3
print("\n" + "=" * 60)
print("TRAINING INCEPTIONV3 (Feature Extraction)")
print("=" * 60)

history_inceptionv3 = inceptionv3_model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    callbacks=get_callbacks('inceptionv3'),
    verbose=1
)

In [ ]:
def plot_training_history(history, model_name):
    """Plot training and validation metrics"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train', linewidth=2)
    axes[0].plot(history.history['val_accuracy'], label='Validation', linewidth=2)
    axes[0].set_title(f'{model_name} - Accuracy', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss
    axes[1].plot(history.history['loss'], label='Train', linewidth=2)
    axes[1].plot(history.history['val_loss'], label='Validation', linewidth=2)
    axes[1].set_title(f'{model_name} - Loss', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'{model_name}_training_history.png', dpi=150, bbox_inches='tight')
    plt.show()

# Plot training histories
plot_training_history(history_cnn, 'Custom_CNN')
plot_training_history(history_resnet, 'ResNet50')
plot_training_history(history_inceptionv3, 'InceptionV3')

In [ ]:
def evaluate_model(model, test_generator, model_name):
    """Comprehensive model evaluation"""
    print(f"\n{'=' * 60}")
    print(f"EVALUATING {model_name}")
    print(f"{'=' * 60}")
    
    # Reset generator
    test_generator.reset()
    
    # Get predictions
    y_pred_proba = model.predict(test_generator, verbose=1)
    y_pred = (y_pred_proba > 0.5).astype(int).flatten()
    y_true = test_generator.classes
    
    # Test loss and accuracy
    test_loss, test_acc = model.evaluate(test_generator, verbose=0)
    
    # Classification Report
    print("\n📊 Classification Report:")
    print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=CLASS_NAMES, yticklabels=CLASS_NAMES,
                annot_kws={'size': 14})
    plt.title(f'{model_name} - Confusion Matrix', fontsize=14, fontweight='bold')
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.tight_layout()
    plt.savefig(f'{model_name}_confusion_matrix.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    return {
        'model_name': model_name,
        'test_accuracy': test_acc,
        'test_loss': test_loss,
        'y_pred': y_pred,
        'y_true': y_true
    }

# %%
# Evaluate all models
results = []

results.append(evaluate_model(custom_cnn, test_generator, 'Custom_CNN'))
results.append(evaluate_model(resnet_model, test_generator, 'ResNet50'))
results.append(evaluate_model(inceptionv3_model, test_generator, 'InceptionV3'))


In [ ]:
# Create comparison dataframe
comparison_df = pd.DataFrame([
    {'Model': r['model_name'], 
     'Test Accuracy': r['test_accuracy'], 
     'Test Loss': r['test_loss']} 
    for r in results
])

comparison_df = comparison_df.sort_values('Test Accuracy', ascending=False)
print("\n📊 MODEL COMPARISON:")
print(comparison_df.to_string(index=False))

# %%
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy comparison
colors = ['#3498db', '#2ecc71', '#e74c3c', '#9b59b6']
bars1 = axes[0].bar(comparison_df['Model'], comparison_df['Test Accuracy'], color=colors)
axes[0].set_ylim(0, 1)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Model Comparison - Test Accuracy', fontsize=14, fontweight='bold')
axes[0].tick_params(axis='x', rotation=45)

for bar in bars1:
    height = bar.get_height()
    axes[0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.2%}', ha='center', va='bottom', fontweight='bold')

# Loss comparison
bars2 = axes[1].bar(comparison_df['Model'], comparison_df['Test Loss'], color=colors)
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].set_title('Model Comparison - Test Loss', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='x', rotation=45)

for bar in bars2:
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.4f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('model_comparison.png', dpi=150, bbox_inches='tight')
plt.show()


In [ ]:
# Find best model
best_model_info = comparison_df.iloc[0]
print(f"\n🏆 BEST MODEL: {best_model_info['Model']}")
print(f"   Test Accuracy: {best_model_info['Test Accuracy']:.2%}")
print(f"   Test Loss: {best_model_info['Test Loss']:.4f}")

# Save all models
custom_cnn.save('models/custom_cnn_final.keras')
resnet_model.save('models/resnet50_final.keras')
inceptionv3_model.save('models/inceptionv3_final.keras')

print("✅ All models saved to 'models/' directory")

# %%
# Save the best model for Streamlit deployment
best_model_name = best_model_info['Model'].lower().replace('_', '')

# Copy best model to deployment folder
import shutil
os.makedirs('streamlit_app', exist_ok=True)

if 'cnn' in best_model_name:
    shutil.copy('models/custom_cnn_final.keras', 'streamlit_app/best_model.keras')
elif 'resnet' in best_model_name:
    shutil.copy('models/resnet50_final.keras', 'streamlit_app/best_model.keras')
elif 'inception' in best_model_name:
    shutil.copy('models/inceptionv3_final.keras', 'streamlit_app/best_model.keras')

print(f"✅ Best model ({best_model_info['Model']}) saved for Streamlit deployment")
